In [ ]:
# Runing on colab with predict-wearer in the google drive
import os
os.chdir('./drive/My Drive/predict-wearer')

In [ ]:
from utilize.data import *
from utilize.model import *
from utilize.transform import *
import tensorflow.keras as keras
import numpy as np

In [ ]:
class ConvLSTM(Model):

    def __init__(self, frame_length = 256, num_channels = 1, p = 0.2, num_classes = 3):

        super(ConvLSTM, self).__init__()

        self.frame_length =  frame_length
        self.num_channels = num_channels
        self.p = p
        self.num_classes = num_classes

        self.conv1 = Conv1D(64, kernel_size = 5, activation = 'relu', input_shape = (frame_length, num_channels))
        self.conv2 = Conv1D(64, kernel_size = 5, activation = 'relu')
        self.conv3 = Conv1D(64, kernel_size = 5, activation = 'relu')
        self.conv4 = Conv1D(64, kernel_size = 5, activation = 'relu')

        self.lstm1 = LSTM(128, return_sequences=True)
        self.lstm2 = LSTM(128, return_sequences=True)

        self.dense = Dense(num_classes, activation = 'softmax')

        self.dropout = Dropout(p)

    def call(self, x):

        x = self.dropout(self.conv1(x))
        x = self.dropout(self.conv2(x))
        x = self.dropout(self.conv3(x))
        x = self.dropout(self.conv4(x))

        x = self.dropout(self.lstm1(x))
        x = self.dropout(self.lstm2(x))

        x = self.dense(x)
        x = x[:, -1, :]

        return x

In [ ]:
DATA_PATH = './data/002_0528.wav'
LALBELS_PATH = './data/002_0528.txt'
LOG_FOLDER = './results/convlstm_multiclass_0604_004_rl0.001_dropout0.4_fl512_hl256_bs128/'

In [ ]:
if not os.path.exists(LOG_FOLDER): 
    os.mkdir(LOG_FOLDER)

In [ ]:
X, y, sr = load_frames_three_class(data_path=DATA_PATH, labels_path=LALBELS_PATH, frame_length=512)

In [ ]:
X.shape, y.shape

((166878, 512, 1), (166878, 3))

In [ ]:
np.unique(y, return_counts=True, axis = 0)

(array([[False, False,  True],
        [False,  True, False],
        [ True, False, False]]), array([118133,  22221,  26524]))

In [ ]:
X_balanced, y_balanced = balance_multi_classes(X, y)
X_train, y_train, X_test, y_test = train_test_split_multi_classes(X_balanced, y_balanced)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((59997, 512, 1), (59997, 3), (6666, 512, 1), (6666, 3))

In [ ]:
model = ConvLSTM(p = 0.4)
model.compile(optimizer = keras.optimizers.RMSprop(learning_rate=0.001),
                loss = 'categorical_crossentropy',
                metrics = ['categorical_accuracy'])

In [ ]:
callback = tf.keras.callbacks.TensorBoard(log_dir = LOG_FOLDER, histogram_freq=1)
model.fit(X_train, y_train, 
          batch_size=128,  epochs=1, verbose = 1, 
          validation_data=(X_test, y_test), 
          callbacks = [callback, keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=20, mode='auto')])

469/469 [==============================] - 43s 92ms/step - loss: 0.9845 - categorical_accuracy: 0.4930 - val_loss: 0.9517 - val_categorical_accuracy: 0.5200
